Code section below for model file

In [1]:
%cd ../flightdelay/utils
from mytrans import MyTrans
import pickle
import pandas as pd
import numpy as np

/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/ronjaboehmer/code/mariajoi/flightdelay/flightdelay/utils


In [4]:
%cd ../data
df = pd.read_csv("130k_zeros_removed.csv")
df = df.drop("Unnamed: 0", axis = 1)

/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/ronjaboehmer/code/mariajoi/flightdelay/flightdelay/data


In [5]:
X = df.drop(columns=['Tail_Number', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay',
             'Distance', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups','CRSDepTime', 'CRSArrTime' , 'Cancelled', 'CancellationCode',
       'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 
        'DestCityName', 'DestState', 'OriginCityName', 'OriginState', 'DestAirportID',
     'Flight_Number_Operating_Airline', 'OriginAirportID','FlightDate','DayofMonth'])

Code section below, not in model file

In [6]:
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.base import TransformerMixin, BaseEstimator

In [7]:
cat_col = make_column_selector(dtype_include=['object'])
ord_col = make_column_selector(dtype_include=['int64'])
cat_transformer = OneHotEncoder(sparse = False, handle_unknown="ignore")
transformer = MyTrans()

preproc_tranformation = make_column_transformer( 
    (cat_transformer, cat_col),
    remainder = "passthrough"
    )


preproc_basic = make_pipeline(transformer, preproc_tranformation)

preproc_basic

Pipeline(steps=[('mytrans', MyTrans()),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12f67afb0>)]))])

In [8]:
%cd ../data
# Export Pipeline as pickle file
with open("preproc_vs1_notfit.pkl", "wb") as file:
    pickle.dump(preproc_basic, file)

Put into model file the below code:

In [ ]:
selected_columns = ['DistanceGroup', 'Month_sin', 'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos']
all_transformed_cols = transformer.transform(X_2k).columns
selected_transformed_cols = all_transformed_cols.intersection(selected_columns)
selected_transformed_cols

In [ ]:
X_transformed = pd.DataFrame(
    preproc_basic.transform(X), 
    columns=np.concatenate([
    preproc_basic.named_steps['columntransformer'].named_transformers_['onehotencoder'].get_feature_names_out(),
    selected_transformed_cols ]))